### Batch download the merged ib1000 from HKMapService and unzip

In [1]:
!wget "https://open.hkmapservice.gov.hk/OpenData/directDownload?productName=iB1000&sheetName=Fullset_Seamless&productFormat=FGDB" -O Fullset_Seamless_FGDB.zip
!unzip -o Fullset_Seamless_FGDB.zip

--2024-08-26 00:05:11--  https://open.hkmapservice.gov.hk/OpenData/directDownload?productName=iB1000&sheetName=Fullset_Seamless&productFormat=FGDB
Resolving open.hkmapservice.gov.hk (open.hkmapservice.gov.hk)... 13.75.70.235
Connecting to open.hkmapservice.gov.hk (open.hkmapservice.gov.hk)|13.75.70.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-zip-compressed]
Saving to: ‘Fullset_Seamless_FGDB.zip’

Fullset_Seamless_FG     [              <=>   ]   1.06G  1.99MB/s    in 9m 1s   

2024-08-26 00:14:15 (2.00 MB/s) - ‘Fullset_Seamless_FGDB.zip’ saved [1135167906]

Archive:  Fullset_Seamless_FGDB.zip
   creating: 20240804/iB1000.gdb/
  inflating: 20240804/iB1000.gdb/a00000001.gdbindexes  
  inflating: 20240804/iB1000.gdb/a00000001.gdbtable  
  inflating: 20240804/iB1000.gdb/a00000001.gdbtablx  
  inflating: 20240804/iB1000.gdb/a00000002.gdbtable  
  inflating: 20240804/iB1000.gdb/a00000002.gdbtablx  
  inflating: 20240804/iB1000.gdb/a

### Install the packages

In [2]:
import os
from google.colab import files
import geopandas as gpd
import fiona
import pandas as pd
from shapely.geometry import Point

### List the Geodatabase

In [9]:
fgdb_path = "/content/20240804/iB1000.gdb"

print(f"Listing layers in {fgdb_path}:")
with fiona.Env():
    for name in fiona.listlayers(fgdb_path):
        print(name)

Listing layers in /content/20240804/iB1000.gdb:
UtilityNumber
BUILDINGRELATEBUILDINGNAME
SITENAME
SITERELATESITENAME
Street_Code
BUILDINGNAME
ADDRESSRELATEBUILDINGSITE
ADDRESS
BSLineAnno
BSPointAnno
BSPolygonAnno
Building
BuildingAccessoryLine
BuildingAnno
BuildingAnnoHouseNo
BuiltStructureLine
BuiltStructurePoint
BuiltStructurePolygon
CartoBuildingLine
ProposedInfraAnno
ProposedInfraStructureLine
ProposedInfraStructurePoint
Site
SiteAnno
SubSite
SubSiteAnno
CartoHydroLine
HydroLine
HydroLineAnno
HydroPoint
HydroPointAnno
HydroPolygon
HydroPolygonAnno
Shoreline
BoulderPoint
BoulderPolygon
CartoLandCoverLine
LandCoverAnno
LandCoverVector2
Tree
PlaceNameAnno
Berm
BMSslope
BMSslopeAnno
CartoReliefLine
Contour
ContourAnno
SpotHeight
VerticalCutPolygon
CartoPedLine
CartoTransLine
PedestrianAnno
RailwayAnno
RailwayEntrance
RailwayPolygon
RoadAnno
RoadAssetPoint
TerminalAnno
TransportLine
TransportLineAnno
TransportPolygon
TransportPolygonAnno
StreetCentreLines
CartoUtilityLine
UtilityLine
Ut

In [10]:
with fiona.Env():
  layer = 'Site'
  with fiona.open(fgdb_path, layer=layer) as src:
    gdf = gpd.GeoDataFrame.from_features(src)
  print(f"CRS: {gdf.crs}")
  print(f"Schema: {gdf.columns.tolist()}")
  print(f"Number of features: {len(gdf)}")

CRS: None
Schema: ['geometry', 'LASTUPDATEDATE', 'SITEID', 'SITESTYPE', 'SITECODE', 'STATUS', 'Shape_Length', 'Shape_Area']
Number of features: 12670


### Extract the feature from the iB1000 Geodatabase

In [13]:
with fiona.Env():
   with fiona.open(fgdb_path, layer=layer) as src:
     gdf = gpd.GeoDataFrame.from_features(src)

   gdf = gdf.set_crs(epsg=2326)
   gdf.to_file("Site_HK1980.geojson", driver='GeoJSON', encoding='utf-8')

In [16]:
with fiona.Env():
   layer = 'SITERELATESITENAME'
   with fiona.open(fgdb_path, layer=layer) as src:
     gdf = gpd.GeoDataFrame.from_features(src)

   gdf = gdf.set_crs(epsg=2326)
   gdf.to_csv("SITERELATESITENAME.csv", index=False, encoding='utf-8')

In [18]:
with fiona.Env():
   layer = 'Tree'
   with fiona.open(fgdb_path, layer=layer) as src:
    gdf = gpd.GeoDataFrame.from_features(src)

   gdf['centroid_x'] = gdf.centroid.x
   gdf['centroid_y'] = gdf.centroid.y
   gdf.to_csv("Tree.csv", index=False, encoding='utf-8')